## Multi Item News Vendor

This script runs SMD and FMD for different n values to compare the solving time (Figure 4).

In [ ]:
import sys

# Add the path to the FairnessML package to the system path
sys.path.append(r'..\..\src\MultiitemNV')
sys.path.append(r'..\..\src\RBTree') 

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
import time
import gurobipy as gp
from gurobipy import GRB
from statistics import mean
from tqdm import tqdm
from MN_test_functions import n_num_test

In [ ]:
current_path = os.getcwd()
parent_path = os.path.dirname(current_path)
parent_path2 = os.path.dirname(parent_path)

In [ ]:
"""
Parameter Setting for the Experiment. Modify the below listed parameters for the experiment.
    n_list_nt: list of n values to run the experiment. This should be the main parameter you may change for the experiment.
    d_nt: number of assets
    K_nt: per-iteration sample size K
    repeats_nt: number of repeats for each n
    warm_start: warm start option for the algorithm
    feas_opt: if 1 solve only a single feasibility problem, if 0 solve feasibility problem until we have objective value that satisfies the objective tolerance
"""

n_list_nt = [3000,5000]

seed_nt = 1234
np.random.seed(seed_nt)
#n_list_nt = np.linspace(10000,20000,11)
d_nt = 10 #Number of assets
beta_nt = 0.1

repeats_nt = 1
K_nt = 200
print_opt = 1
warm_start = 0 #0: no warm start, 1: warm start
feas_opt =  0

SMD_run_time_arr = np.zeros(repeats_nt)
FMD_run_time_arr = np.zeros(repeats_nt)
SMD_real_T_arr = np.zeros(repeats_nt)
FMD_real_T_arr = np.zeros(repeats_nt)
SMD_T_arr = np.zeros(repeats_nt)
FMD_T_arr = np.zeros(repeats_nt)
SMD_dualgap_time = np.zeros(repeats_nt)
FMD_dualgap_time = np.zeros(repeats_nt)
for n_num in n_list_nt:
    for rep_idx in range(repeats_nt):
        stat_SMD, stat_FMD, _ = n_num_test(int(n_num), d_nt, beta_nt, K_nt, print_opt = 1, feas_opt = feas_opt, warm_start=warm_start)
        SMD_run_time_arr[rep_idx] = stat_SMD.total_solved_time
        FMD_run_time_arr[rep_idx] = stat_FMD.total_solved_time
        SMD_real_T_arr[rep_idx] = stat_SMD.real_T_list[0]
        FMD_real_T_arr[rep_idx] = stat_FMD.real_T_list[0]
        SMD_T_arr[rep_idx] = stat_SMD.max_iter
        FMD_T_arr[rep_idx] = stat_FMD.max_iter
        SMD_dualgap_time[rep_idx] = stat_SMD.dual_gap_time
        FMD_dualgap_time[rep_idx] = stat_FMD.dual_gap_time
    data_arr = np.vstack(
        (SMD_run_time_arr, SMD_real_T_arr, SMD_T_arr,SMD_dualgap_time, FMD_run_time_arr, FMD_real_T_arr, FMD_T_arr,
        FMD_dualgap_time)).T
    df = pd.DataFrame(data_arr, columns=['SMD_Solve_time', 'SMD_Iter', 'SMD_Max_Iter','SMD_dual_time', \
                                         'FMD_Solve_time', 'FMD_Iter', 'FMD_Max_Iter','FMD_dual_time'])

    custom_path = '/results/MultiitemNV/n_num_result/d={}_n={}_K={}.csv'.format(d_nt, n_num, K_nt)
    save_path = parent_path2 + custom_path
    df.to_csv(save_path, index=False)

    """
    df contains the following columns:
        Columns Regarding SMD:
            SMD_Solve_time: Total time taken to solve the problem using SMD
            SMD_Iter: Total number of iterations taken to solve the problem using SMD
            SMD_Max_Iter: Maximum number of iterations for SMD
            SMD_dual_time: Total time Computing Duality gap for SMD
        Columns Regarding FMD:
            FMD_Solve_time: Total time taken to solve the problem using FMD
            FMD_Iter: Total number of iterations taken to solve the problem using FMD
            FMD_Max_Iter: Maximum number of iterations for FMD
            FMD_dual_time: Total time Computing Duality gap for FMD
    """